## 11_13 - Python Finance

**Capitulo 11**: O site INVESTING.COM possui uma imensa base de dados de ativos financeiros de vários países.

Objetivo: acessar a base de dados do investing.com com python para obter dados de cotas de fundos de investimento brasileiros. Como exemplo, comparamos o fundo alaska black, ip participações e bova11

**Capitulo 12**: O site INVESTING.COM possui uma imensa base de dados de ativos financeiros de vários países.

Objetivo: Acessar a base de dados do investing.Com com python. Vamos obter cotações de empresas brasileiras e índices de ações como  o índice bovespa e o índice de fundos imobiliários ifix.

**Capitulo 13**: O site INVESTING.COM possui uma imensa base de dados de ativos financeiros de vários países.

Objetivo: Montar estrutura a termo da curva de juros com dados do investing.com utilizando python.

## 1. Importando bibliotecas

In [ ]:
!pip install investpy

In [ ]:
import investpy as inv

In [ ]:
import plotly.graph_objects as go

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (16,8)

##2. Fundos de Investimentos

In [ ]:
fundos_br = inv.get_funds_list(country='brazil')
fundos_br

In [ ]:
len(fundos_br)

In [ ]:
fundos_br[:5]

In [ ]:
pesq = inv.search_funds(by='name', value='alaska black')
pesq

In [ ]:
pesq['name'][0]

In [ ]:
fundo = 'Alaska Black Fundo De Investimento Em Cotas De Fundos De Investimento Em Ações - Bdr Nível I'
alaska = inv.get_fund_historical_data(fundo, country='brazil', from_date='01/01/1900', to_date='26/04/2020')['Close']
alaska

In [ ]:
alaska.plot();

In [ ]:
pesq2 = inv.search_funds(by='name', value='ip parti')
pesq2

In [ ]:
pesq2['name'][1]

In [ ]:
fundo = 'Ip Participações Ipg Fundo De Investimento Em Cotas De Fundos De Investimento Em Ações Bdr Nível I'
ip = inv.get_fund_historical_data(fundo, country='brazil', from_date='01/01/1900', to_date='26/04/2020')['Close']
ip

In [ ]:
ip.plot();

In [ ]:
inv.get_etfs_overview(country='brazil')

In [ ]:
bova11 = inv.get_etf_historical_data('Ishares Ibovespa', country='brazil', from_date='01/01/1900', to_date='25/04/2020')['Close']
bova11

In [ ]:
bova11.plot();

In [ ]:
comparativo = pd.DataFrame()
data_inicio = ip.index[0]

comparativo['ip'] = ip / ip.loc[data_inicio]
comparativo['alaska'] = alaska / alaska.loc[data_inicio]
comparativo['bova11'] = bova11 / bova11.loc[data_inicio]

comparativo

In [ ]:
comparativo.plot();

## 3. Índices

In [ ]:
inv.get_indices_list('brazil')

In [ ]:
inv.get_index_recent_data('Bovespa', country='brazil')

In [ ]:
ibov = inv.get_index_historical_data('Bovespa', country='brazil', from_date='01/01/1900', to_date='26/04/2020')
ibov

In [ ]:
ibov_dolarizado = inv.get_index_historical_data('Ibovespa USD', country='brazil', from_date='01/01/1900', to_date='26/04/2020')
ibov_dolarizado

In [ ]:
ifix = inv.get_index_historical_data('BM&FBOVESPA Real Estate IFIX', country='brazil', from_date='01/01/1900', to_date='26/04/2020')
ifix

In [ ]:
indices = pd.DataFrame()
data_inicio = ifix.index[0]

indices['ifix'] = ifix['Close'] / ifix['Close'].loc[data_inicio]
indices['ibov'] = ibov['Close'] / ibov['Close'].loc[data_inicio]

In [ ]:
indices.plot();

## 4. Ações

In [ ]:
inv.get_stocks('brazil')

In [ ]:
inv.get_stock_company_profile('bbdc4', country='brazil')

In [ ]:
bradesco = inv.get_stock_historical_data('bbdc4', country='brazil',  from_date='01/01/1900', to_date='25/04/2020')

In [ ]:
bradesco['Close'].plot();

In [ ]:
inv.get_stock_information('bbdc4', country='brazil')

In [ ]:
inv.technical_indicators('bbdc4', country='brazil', product_type='stock')

In [ ]:
inv.moving_averages('bbdc4', country='brazil', product_type='stock')

## 5. Curva de Juros

In [ ]:
bondsBR = inv.get_bonds_list('brazil')

In [ ]:
bondsBR

In [ ]:
bonds_overview  = inv.get_bonds_overview('brazil')

In [ ]:
bonds_overview

In [ ]:
plt.plot(bondsBR, bonds_overview['last_close']);

In [ ]:
data_inicio = '01/01/2010'
data_fim = '28/04/2020'

um_ano = inv.get_bond_historical_data('Brazil 1Y', from_date=data_inicio, to_date=data_fim)
um_ano

In [ ]:
um_ano['Close'].plot();

In [ ]:
data_inicio = '01/01/2010'
data_fim = '28/04/2020'
bonds = pd.DataFrame()

for prazo in bondsBR:
  bonds[prazo] = inv.get_bond_historical_data(prazo, from_date=data_inicio, to_date=data_fim)['Close']

In [ ]:
bonds.index = pd.to_datetime(bonds.index)

In [ ]:
bonds

In [ ]:
plt.plot(bondsBR , bonds.loc['2020-04-23'].values)

In [ ]:
fig = go.Figure()

for i in bonds.index:
  fig.add_trace(go.Scatter(x=bondsBR, y=bonds.loc[i], mode='lines', name=str(i)))

fig.show()

In [ ]:
data_inicio = '01/01/2010'
data_fim = '28/04/2020'
bondsMensal = pd.DataFrame()

for prazo in bondsBR:
  bondsMensal[prazo] = inv.get_bond_historical_data(prazo, from_date=data_inicio, to_date=data_fim, interval='Monthly')['Close']

In [ ]:
fig = go.Figure()

for i in bondsMensal.index:
  fig.add_trace(
      go.Scatter(
          x=bondsBR, 
          y=bondsMensal.loc[i], 
          mode='lines', 
          name=str(i),
          visible=False
      )
  )

fig.data[0].visible = True

steps = []

for i in range(len(fig.data)):
  step = dict(
      method='restyle',
      args=['visible', [False] * len(fig.data)],
      label=fig.data[i]['name'][:7]
  )
  step['args'][1][i] = True
  steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={'prefix':"Mês: "},
    pad={'t':50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    yaxis = dict(range=[3,14.5])
)

fig.show()

In [ ]:
def consulta_bc(codigo_bcb):
  url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
  df = pd.read_json(url)
  df['data'] = pd.to_datetime(df['data'], dayfirst=True)
  df.set_index('data', inplace=True)
  return df

In [ ]:
selic_meta = consulta_bc(432)

In [ ]:
s2015 = selic_meta[selic_meta.index.year >= 2015]
b2015 = bonds[bonds.index.year >= 2015]

In [ ]:
fig = go.Figure()

for i in bondsBR:
  fig.add_trace(go.Scatter(x=b2015.index, y=b2015[i], mode='lines', name=str(i)))

fig.add_trace(go.Scatter(x=s2015.index, y=s2015['valor'], mode='lines', name='SELIC', line={'color':'black'}))

fig.show()